In [1]:
import tensorflow as tf
import numpy as np

d:\develop\python\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\develop\python\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\develop\python\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\develop\python\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a

In [2]:
tf.__version__

'2.0.0-alpha0'

In [3]:
tf.test.is_gpu_available()

True

In [4]:
v = tf.Variable(0.0)

In [5]:
v

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.0>

In [6]:
(v+1).numpy()

1.0

In [7]:
v.assign(5)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=5.0>

In [8]:
v

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=5.0>

In [9]:
v.assign_add(1)
v

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=6.0>

In [10]:
v.read_value()

<tf.Tensor: id=26, shape=(), dtype=float32, numpy=6.0>

In [11]:
w = tf.Variable([[1.0]])
with tf.GradientTape() as t:
    loss = w * w

In [12]:
grad = t.gradient(loss,w)

In [13]:
grad

<tf.Tensor: id=43, shape=(1, 1), dtype=float32, numpy=array([[2.]], dtype=float32)>

In [14]:
w = tf.constant(3.0)
with tf.GradientTape() as t:
    t.watch(w)
    loss = w * w

In [15]:
d_loss_dw = t.gradient(loss,w)

In [16]:
d_loss_dw

<tf.Tensor: id=50, shape=(), dtype=float32, numpy=6.0>

In [17]:
w = tf.constant(3.0)
with tf.GradientTape(persistent=True) as t:
    t.watch(w)
    y = w * w
    z = y * y

In [18]:
dy_dw = t.gradient(y,w)
dy_dw

<tf.Tensor: id=58, shape=(), dtype=float32, numpy=6.0>

In [19]:
dz_dw = t.gradient(z,w)
dz_dw

<tf.Tensor: id=66, shape=(), dtype=float32, numpy=108.0>

# 自定义梯度下降

In [68]:
(train_image,train_labels),(test_image,test_labels) = tf.keras.datasets.mnist.load_data()

In [69]:
train_image = tf.expand_dims(train_image,-1)
test_image = tf.expand_dims(test_image,-1)

In [104]:
train_labels.shape
# test_labels.shape

TensorShape([60000])

In [106]:
# train_image.shape
test_image.shape


TensorShape([10000, 28, 28, 1])

In [107]:
train_image = tf.cast(train_image/255,tf.float32)
test_image = tf.cast(test_image/255,tf.float32)

In [108]:
train_labels = tf.cast(train_labels,tf.int64)
test_labels = tf.cast(test_labels,tf.int64)

In [109]:
dataset = tf.data.Dataset.from_tensor_slices((train_image, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_image, test_labels))

In [110]:
dataset
test_dataset

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

In [111]:
dataset = dataset.shuffle(10000).batch(32)
test_dataset = test_dataset.batch(32)

In [112]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,[3,3],activation='relu',input_shape = (None,None,1)),
    tf.keras.layers.Conv2D(32,[3,3],activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(10)
])

In [113]:
optimizer = tf.keras.optimizers.Adam()

In [114]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [115]:
features, labels = next(iter(dataset))

In [116]:
features.shape

TensorShape([32, 28, 28, 1])

In [117]:
labels.shape

TensorShape([32])

In [118]:
predictions = model(features)

In [119]:
predictions.shape

TensorShape([32, 10])

In [120]:
tf.argmax(predictions,axis=1)

<tf.Tensor: id=1982927, shape=(32,), dtype=int64, numpy=
array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int64)>

In [121]:
labels

<tf.Tensor: id=1982909, shape=(32,), dtype=int64, numpy=
array([7, 7, 4, 9, 5, 7, 2, 5, 9, 3, 2, 0, 5, 7, 7, 5, 0, 7, 4, 1, 2, 8,
       5, 2, 6, 6, 3, 5, 3, 9, 2, 0], dtype=int64)>

In [122]:
def loss(model,x,y):
    y_ = model(x)
    return loss_func(y,y_)


In [123]:
train_loss = tf.keras.metrics.Mean('train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

test_loss = tf.keras.metrics.Mean('train_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [124]:
def train_step(model, images, labels):
    with tf.GradientTape() as t:
        pred = model(images)
        loss_step = loss(model,images,labels)
    grads = t.gradient(loss_step, model.trainable_variables)
    optimizer.apply_gradients(zip(grads,model.trainable_variables))
    train_loss(loss_step)
    train_accuracy(labels, pred)

In [125]:
def test_step(model, images, labels):
    pred = model(images)
    loss_step = loss(model,images,labels)
    test_loss(loss_step)
    test_accuracy(labels, pred)

In [128]:
def train():
    for epoch in range(10):
        for (batch,(images, labels)) in enumerate(dataset):
            train_step(model,images,labels)
        print('Epoch{} lossis {},accuracy is {}'.format(epoch,train_loss.result(),train_accuracy.result()))
        
        for (batch,(images, labels)) in enumerate(test_dataset):
            test_step(model,images,labels)
        print('Epoch{} test_loss is {},accuracy is {}'.format(epoch,test_loss.result(),test_accuracy.result()))
        
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

In [129]:
train()

Epoch0 lossis 2.3014519214630127,accuracy is 0.11198333650827408
Epoch0 test_loss is 2.301251173019409,accuracy is 0.11349999904632568
Epoch1 lossis 2.3013429641723633,accuracy is 0.11236666887998581
Epoch1 test_loss is 2.3012478351593018,accuracy is 0.11349999904632568


KeyboardInterrupt: 

# metrics汇总计算模块

In [47]:
m = tf.keras.metrics.Mean('acc')

In [48]:
m(20)

<tf.Tensor: id=650764, shape=(), dtype=float32, numpy=20.0>

In [49]:
m(10)

<tf.Tensor: id=650775, shape=(), dtype=float32, numpy=15.0>

In [51]:
m.result().numpy()

15.0

In [52]:
m([30,40])

<tf.Tensor: id=650796, shape=(), dtype=float32, numpy=25.0>

In [53]:
m.reset_states()

In [54]:
m(1)

<tf.Tensor: id=650809, shape=(), dtype=float32, numpy=1.0>

In [55]:
a = tf.keras.metrics.SparseCategoricalAccuracy('acc')

In [56]:
a(labels,model(features))

<tf.Tensor: id=650854, shape=(), dtype=float32, numpy=0.5625>